## IMPORT LIBRARY

In [ ]:
# pip uninstall nltk

In [ ]:
pip install -U nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd

import math as m
import nltk
import re
import string
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer as cv
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## IMPORT DATASET

In [ ]:
df = pd.read_csv("E-Commerce Dataset.csv", sep=",")
df.head()

,SL,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Num_reviews,Avg_rating,Std_dev_rating,Num_recommends,Percent_recommends,Percent_num_reviews,Rating_strength,Item_success,Division Name,Department Name,Class Name
0,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,43,3.98,1.171,34,0.791,0.009,0.00021,Not Trending,General,Dresses,Dresses
1,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,9,4.78,0.629,8,0.889,0.002,0.00005,Not Trending,General Petite,Bottoms,Pants
2,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,3,4.67,0.471,3,1.000,0.001,0.00001,Not Trending,General,Tops,Blouses
3,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,87,4.39,0.939,74,0.851,0.017,0.00165,Trending,General,Dresses,Dresses
4,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,2,4.50,0.500,2,1.000,0.000,0.00000,Not Trending,General Petite,Tops,Knits


In [ ]:
dataset = df[['Clothing ID', 'Age', 'Class Name', 'Department Name', 'Division Name', 'Rating', \
'Recommended IND', 'Review Text', 'Title', 'Item_success']]

## DATA CHECKING

In [ ]:
# Check whether there's null value in each feature

dataset.isnull().sum()

Clothing ID        0
Age                0
Class Name         0
Department Name    0
Division Name      0
Rating             0
Recommended IND    0
Review Text        0
Title              0
Item_success       0
dtype: int64

## SENTIMENT ANALYSIS DATASET

### 1. Remove Digits

In [ ]:
def remove_digits(text):
    digits = [str(x) for x in range(10)]
    for d in digits:
        text = text.replace(d, ' ')
    return text

In [ ]:
# Deleting digits in text:

dataset['Review Text'] = dataset['Review Text'].apply(lambda x: remove_digits(x))
dataset['Title'] = dataset['Title'].apply(lambda x: remove_digits(x))
dataset.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Clothing ID,Age,Class Name,Department Name,Division Name,Rating,Recommended IND,Review Text,Title,Item_success
0,1077,60,Dresses,Dresses,General,3,0,I had such high hopes for this dress and reall...,Some major design flaws,Not Trending
1,1049,50,Pants,Bottoms,General Petite,5,1,"I love, love, love this jumpsuit. it's fun, fl...",My favorite buy!,Not Trending
2,847,47,Blouses,Tops,General,5,1,This shirt is very flattering to all due to th...,Flattering shirt,Not Trending
3,1080,49,Dresses,Dresses,General,2,0,"I love tracy reese dresses, but this one is no...",Not for the very petite,Trending
4,858,39,Knits,Tops,General Petite,5,1,I aded this in my basket at hte last mintue to...,Cagrcoal shimmer fun,Not Trending


### 2. Remove Punctuation

In [ ]:
string.punctuation
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_no_punct=''.join(no_punct)
    return words_no_punct

In [ ]:
# Apply remove punctuation function above on each data:

dataset['Review Text'] = dataset['Review Text'].apply(lambda x: remove_punctuation(x))
dataset['Title'] = dataset['Title'].apply(lambda x: remove_punctuation(x))
dataset.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Clothing ID,Age,Class Name,Department Name,Division Name,Rating,Recommended IND,Review Text,Title,Item_success
0,1077,60,Dresses,Dresses,General,3,0,I had such high hopes for this dress and reall...,Some major design flaws,Not Trending
1,1049,50,Pants,Bottoms,General Petite,5,1,I love love love this jumpsuit its fun flirty ...,My favorite buy,Not Trending
2,847,47,Blouses,Tops,General,5,1,This shirt is very flattering to all due to th...,Flattering shirt,Not Trending
3,1080,49,Dresses,Dresses,General,2,0,I love tracy reese dresses but this one is not...,Not for the very petite,Trending
4,858,39,Knits,Tops,General Petite,5,1,I aded this in my basket at hte last mintue to...,Cagrcoal shimmer fun,Not Trending


### 3. Tokenization

In [ ]:
def tokenizer(text):
    tokenizer = WordPunctTokenizer()
    return tokenizer.tokenize(text)

In [ ]:
# Apply tokenize string values:

dataset['Review Text'] = dataset['Review Text'].apply(lambda x: tokenizer(x))
dataset['Title'] = dataset['Title'].apply(lambda x: tokenizer(x))
dataset.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Clothing ID,Age,Class Name,Department Name,Division Name,Rating,Recommended IND,Review Text,Title,Item_success
0,1077,60,Dresses,Dresses,General,3,0,"[I, had, such, high, hopes, for, this, dress, ...","[Some, major, design, flaws]",Not Trending
1,1049,50,Pants,Bottoms,General Petite,5,1,"[I, love, love, love, this, jumpsuit, its, fun...","[My, favorite, buy]",Not Trending
2,847,47,Blouses,Tops,General,5,1,"[This, shirt, is, very, flattering, to, all, d...","[Flattering, shirt]",Not Trending
3,1080,49,Dresses,Dresses,General,2,0,"[I, love, tracy, reese, dresses, but, this, on...","[Not, for, the, very, petite]",Trending
4,858,39,Knits,Tops,General Petite,5,1,"[I, aded, this, in, my, basket, at, hte, last,...","[Cagrcoal, shimmer, fun]",Not Trending


### 4. Remove Stopwords

In [ ]:
nltk.download('stopwords')
stops_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(tokens, is_lower_case=False):
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stops_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stops_list]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
# Apply removing stopwords function:

dataset['Review Text'] = dataset['Review Text'].apply(lambda x: remove_stopwords(x))
dataset['Title'] = dataset['Title'].apply(lambda x: remove_stopwords(x))
dataset.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Clothing ID,Age,Class Name,Department Name,Division Name,Rating,Recommended IND,Review Text,Title,Item_success
0,1077,60,Dresses,Dresses,General,3,0,high hopes dress really wanted work initially ...,major design flaws,Not Trending
1,1049,50,Pants,Bottoms,General Petite,5,1,love love love jumpsuit fun flirty fabulous ev...,favorite buy,Not Trending
2,847,47,Blouses,Tops,General,5,1,shirt flattering due adjustable front tie perf...,Flattering shirt,Not Trending
3,1080,49,Dresses,Dresses,General,2,0,love tracy reese dresses one petite feet tall ...,petite,Trending
4,858,39,Knits,Tops,General Petite,5,1,aded basket hte last mintue see would look lik...,Cagrcoal shimmer fun,Not Trending


### 5. Stemming

In [ ]:
porter = PorterStemmer()

In [ ]:
def stemming(words):
    new_words = [porter.stem(w) for w in words]
    return new_words

In [ ]:
# Apply stemming function:
dataset['Review Text'] = dataset['Review Text'].apply(lambda x: stemming(x.split()))
dataset['Title'] = dataset['Title'].apply(lambda x: stemming(x.split()))
dataset.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Clothing ID,Age,Class Name,Department Name,Division Name,Rating,Recommended IND,Review Text,Title,Item_success
0,1077,60,Dresses,Dresses,General,3,0,"[high, hope, dress, realli, want, work, initi,...","[major, design, flaw]",Not Trending
1,1049,50,Pants,Bottoms,General Petite,5,1,"[love, love, love, jumpsuit, fun, flirti, fabu...","[favorit, buy]",Not Trending
2,847,47,Blouses,Tops,General,5,1,"[shirt, flatter, due, adjust, front, tie, perf...","[flatter, shirt]",Not Trending
3,1080,49,Dresses,Dresses,General,2,0,"[love, traci, rees, dress, one, petit, feet, t...",[petit],Trending
4,858,39,Knits,Tops,General Petite,5,1,"[ade, basket, hte, last, mintu, see, would, lo...","[cagrcoal, shimmer, fun]",Not Trending


## VECTORIZER

In [ ]:
features_RC = dataset[['Clothing ID', 'Review Text']]

In [ ]:
features_RC['Review Text'] = features_RC['Review Text'].apply(lambda x: ' '.join([str(word) for word in x]))
features_RC = features_RC.to_numpy().reshape(-1, 2)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
label = dataset['Rating'].to_numpy().reshape(-1, 1)
label

array([[3],
       [5],
       [5],
       ...,
       [5],
       [5],
       [5]])

# Split Into Training and Testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features_RC, label, test_size=0.1, random_state=12) # Test data 1%, Train data 90%

### 1. Count Vectorizer

In [ ]:
x_train[:, 1]

array(['want buy dress everyth love bought two neutral black turquois turquois one retro print get mani compliment comfort flatter im price point retail pleas',
       'dresss much shorter describ see thru though guess tell pictur expect price',
       'tri top store didnt want buy full price even though made heart skip beat ador even cuter saw final went sale got addit immedi order flowi comfort fun wore today layer black turtleneck long black vest',
       ...,
       'cloth stone never let awesom qualiti comfort versatil imagin live dress summer soft fabric easi breezi fit pocket machin washablewhat love go buy',
       'love dress wear equival wear coziest tshirt stretch top chest need necklin modest stripe geometr pattern flatter vibrant gaudi slightest length perfect shortest part knee silhouett tight natur waist flare flowi flatter way got larg fluctuat lb',
       'love layer basic tee graphic one like wild west tee add right amount edg current build pin collect add vintag turq

In [ ]:
vector = cv(max_features=3000, ngram_range=(1, 3))

In [ ]:
xcv_train = vector.fit_transform(x_train[:, 1])
xcv_test = vector.transform(x_test[:, 1])

### 2. TF-IDF Vectorizer

In [ ]:
tf = tfidf(ngram_range =(1,3),max_features = 3000)
xtf_train = tf.fit_transform(x_train[:, 1])
xtf_test = tf.transform(x_test[:, 1])

# MULTI-LABEL CLASSIFICATION

In [ ]:
# Using Supprt Vector Machine Algorithm:

from sklearn.multiclass import OneVsRestClassifier as ovr
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

LinearSVC Classifier

In [ ]:
clf_sv = LinearSVC(class_weight='balanced', multi_class='ovr')
clf = clf_sv.fit(xtf_train, y_train)
svc_pred = clf.predict(xtf_test)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
accuracy_score(y_test, svc_pred)

0.578

### SVM Classifier

1. CountVectorizer() Features

In [ ]:
clf_cv = ovr(SVC()).fit(xcv_train, y_train)
cv_pred = clf_cv.predict(xcv_test)

In [ ]:
print(f"SVM Classifier using CountVectorizer(): {accuracy_score(y_test, cv_pred)}") 

SVM Classifier using CountVectorizer(): 0.634


2. TF-IDF Vectorizer()

In [ ]:
# clf_tf = ovr(SVC()).fit(xtf_train, y_train)
# tf_pred = clf_cv.predict(xtf_test)

In [ ]:
# accuracy_score(y_test, tf_pred)

In [ ]:
# cv_train = clf_cv.predict(xcv_train)
# accuracy_score(y_train, cv_train)

Concatinate features

In [ ]:
cv_pred = cv_pred.reshape(-1, 1)

In [ ]:
dt = pd.DataFrame(np.concatenate([x_test, y_test, cv_pred], axis=1), \
columns=['ID', 'Review Text', 'Rating Actual', 'Rating Prediction'])
dt

,ID,Review Text,Rating Actual,Rating Prediction
0,877,bought store yesterday light fit extrem comfor...,5,5
1,1080,color dress beauti realli want love wasnt cut ...,3,3
2,964,order littl jacket knew would like upon arriv ...,5,5
3,1060,pound bought p fabric high qualiti cling inste...,5,5
4,872,nice casual inexpens shirt lace necklin greatn...,4,5
...,...,...,...,...
495,829,shirt perfect goto classic white button twist ...,5,5
496,877,featur pictur im feet lb order xs petit im pre...,5,5
497,1081,love dress fabric super soft flowi awar model ...,4,5
498,1094,excit order dress perfect red shoe go unfortun...,2,5


## PREDICT SUCCESS ITEM

In [ ]:
dt['Rating Prediction'] = dt['Rating Prediction'].astype(int)
dt['Rating Actual'] = dt['Rating Actual'].astype(int)

In [ ]:
# CALCULATE AVERAGE AND STANDARD DEVIATION ON 'RATING PREDICTION'

rpi1 = dt.groupby('ID').agg({'ID': 'size', 'Rating Prediction': ['mean', 'std']})
rpi1 = rpi1.reset_index()
rpi1 = rpi1.fillna(0)
rpi1.columns = rpi1.columns.droplevel(0)
rpi1.columns = [['Clothing ID', 'Clothing Count', 'Rating Mean', 'Rating Std']]
rpi1

,Clothing ID,Clothing Count,Rating Mean,Rating Std
0,35,1,5.000000,0.00000
1,107,1,5.000000,0.00000
2,121,1,4.000000,0.00000
3,144,1,5.000000,0.00000
4,154,4,4.750000,0.50000
...,...,...,...,...
160,1125,4,5.000000,0.00000
161,1126,3,4.666667,0.57735
162,1131,1,5.000000,0.00000
163,1145,1,5.000000,0.00000


In [ ]:
# CALCULATE AVERAGE AND STANDARD DEVIATION ON 'RATING ACTUAL'

rpi2 = dt.groupby('ID').agg({'ID': 'size', 'Rating Actual': ['mean', 'std']})
rpi2 = rpi2.reset_index()
rpi2 = rpi2.fillna(0)
rpi2.columns = rpi2.columns.droplevel(0)
rpi2.columns = [['Clothing ID', 'Clothing Count', 'Rating Mean', 'Rating Std']]
rpi2

,Clothing ID,Clothing Count,Rating Mean,Rating Std
0,35,1,5.000000,0.000000
1,107,1,4.000000,0.000000
2,121,1,4.000000,0.000000
3,144,1,5.000000,0.000000
4,154,4,3.750000,1.258306
...,...,...,...,...
160,1125,4,4.500000,0.577350
161,1126,3,4.333333,1.154701
162,1131,1,3.000000,0.000000
163,1145,1,5.000000,0.000000


In [ ]:
def find_strength(dt, rpi, factor):
    for idx in dt:
        for i in range(len(rpi)):
            if(idx == rpi[i, 0]):   
                proportion = rpi[i, 1] / len(rpi)
                num = (proportion * rpi[i, 2]) / rpi[i, 3]
                rating = (1 - m.exp(num)) / (1 + m.exp(num))
                strength = rating ** 2
                # print(f"{idx} | {rpi[i, 0]}\t|{strength}")
                factor.append(strength)
                break
    return factor

In [ ]:
factor1 = []
factor2 = []
rpi1 = rpi1.to_numpy()
rpi2 = rpi2.to_numpy()

factor1 = find_strength(dt['ID'], rpi1, factor1)
factor2 = find_strength(dt['ID'], rpi2, factor2)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


In [ ]:
dt['Strength Prediction'] = factor1
dt['Strength Actual'] = factor2
dt['Strength Prediction'] = dt['Strength Prediction'].fillna(0)
dt['Strength Actual'] = dt['Strength Actual'].fillna(0)

In [ ]:
dt

,ID,Review Text,Rating Actual,Rating Prediction,Strength Prediction,Strength Actual
0,877,bought store yesterday light fit extrem comfor...,5,5,0.000000,0.000000
1,1080,color dress beauti realli want love wasnt cut ...,3,3,0.014121,0.005380
2,964,order littl jacket knew would like upon arriv ...,5,5,0.000000,0.013144
3,1060,pound bought p fabric high qualiti cling inste...,5,5,0.000000,0.000000
4,872,nice casual inexpens shirt lace necklin greatn...,4,5,0.042172,0.030463
...,...,...,...,...,...,...
495,829,shirt perfect goto classic white button twist ...,5,5,0.143644,0.043667
496,877,featur pictur im feet lb order xs petit im pre...,5,5,0.000000,0.000000
497,1081,love dress fabric super soft flowi awar model ...,4,5,0.459111,0.109570
498,1094,excit order dress perfect red shoe go unfortun...,2,5,0.105568,0.023460


In [ ]:
def result(number):
    if number > cutoff:
        return "Trending"
    else:
        return "Not Trending"

In [ ]:
cutoff = 0.00037 # If above the cutoff --> trending, If not above the cutoff --> not trending
dt['Success Prediction'] = dt['Strength Prediction'].apply(lambda x: result(x))
dt['Success Actual'] = dt['Strength Actual'].apply(lambda x: result(x))

In [ ]:
dt

,ID,Review Text,Rating Actual,Rating Prediction,Strength Prediction,Strength Actual,Success Prediction,Success Actual
0,877,bought store yesterday light fit extrem comfor...,5,5,0.000000,0.000000,Not Trending,Not Trending
1,1080,color dress beauti realli want love wasnt cut ...,3,3,0.014121,0.005380,Trending,Trending
2,964,order littl jacket knew would like upon arriv ...,5,5,0.000000,0.013144,Not Trending,Trending
3,1060,pound bought p fabric high qualiti cling inste...,5,5,0.000000,0.000000,Not Trending,Not Trending
4,872,nice casual inexpens shirt lace necklin greatn...,4,5,0.042172,0.030463,Trending,Trending
...,...,...,...,...,...,...,...,...
495,829,shirt perfect goto classic white button twist ...,5,5,0.143644,0.043667,Trending,Trending
496,877,featur pictur im feet lb order xs petit im pre...,5,5,0.000000,0.000000,Not Trending,Not Trending
497,1081,love dress fabric super soft flowi awar model ...,4,5,0.459111,0.109570,Trending,Trending
498,1094,excit order dress perfect red shoe go unfortun...,2,5,0.105568,0.023460,Trending,Trending


# EVALUATION

In [ ]:
from sklearn.preprocessing import LabelEncoder

xf = dt['Success Prediction']
yf = dt['Success Actual']

In [ ]:
encoder = LabelEncoder()
xf = np.array(xf).reshape(-1, 1)
xf = encoder.fit_transform(xf)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
yf = np.array(yf).reshape(-1, 1)
yf = encoder.fit_transform(yf)

In [ ]:
yf

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,

In [ ]:
xf

array([0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,

In [ ]:
p = np.subtract(xf, yf)
p = p.reshape(-1, 1)

MODEL ACCURACY

In [ ]:
from sklearn import metrics

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yf, xf)))

Root Mean Squared Error: 0.4289522117905443


In [ ]:
print('Accuracy Score:', np.sqrt(metrics.accuracy_score(yf, xf)))

Accuracy Score: 0.9033271832508971


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9ed8929e-5e1a-4a7a-9cf6-aa58624debce' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>